# Experience

## Import module

In [1]:
from utils import BasicDataset
from unet import UNet
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.scenarios.dataset_scenario import benchmark_from_datasets
from avalanche.evaluation.metrics import WeightCheckpoint, gpu_usage_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, disk_usage_metrics
from avalanche.benchmarks.utils import AvalancheDataset
from avalanche.logging import InteractiveLogger, TextLogger, WandBLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.supervised import Naive,EWC
from avalanche.benchmarks.scenarios.supervised import class_incremental_benchmark
from torch import optim
import numpy as np
import wandb

## Prepare Dataset

In [2]:
from avalanche.benchmarks import benchmark_from_datasets
from avalanche.benchmarks.utils.data_attribute import DataAttribute
import torch

In [3]:
aval_datasets = []
for i in range(5):
    train_dataset = BasicDataset(images_dir=f"./data/train/img/{i}",
                                mask_dir=f"./data/train/label/{i}")
    aval_dataset = AvalancheDataset([train_dataset],
                                    data_attributes=[
                                                    DataAttribute([0] * len(train_dataset), "targets_task_labels"),
                                                    ])
    aval_datasets.append(aval_dataset)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:02<00:00, 21.27it/s]


In [4]:
bm = benchmark_from_datasets(train=aval_datasets,
                             test=[aval_datasets[0]]*len(aval_datasets))

In [5]:
print(f"original training samples = {len(bm.train_stream[0].dataset)}")

original training samples = 51


## Settings

In [6]:
epochs = 1
batch_size= 8
learning_rate = 0.00001
weight_decay = 0.00000001
momentum = 0.999

## Train using NAIVE

In [7]:
func_name = "NAIVE"

In [8]:
model = UNet(n_channels=3, n_classes=4)

In [9]:

# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger(f"./log/tb_data/{func_name}")

wandb_logger = WandBLogger(project_name="UNet_CL",
                           run_name=func_name,
                           path=f"./log/checkpoint/{func_name}",
                           dir=f"./log/wandb/{func_name}")

# log to text file
text_logger = TextLogger(open(f'./log/{func_name}.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: boyifan1 (kirigiri_suzumiya). Use `wandb login --relogin` to force relogin
wandb: WARNING Path ./log/wandb/NAIVE\wandb\ wasn't writable, using system temp directory.


In [10]:
# CREATE THE STRATEGY INSTANCE (NAIVE)
optimizer = optim.RMSprop(model.parameters(),
                          lr=learning_rate, 
                          weight_decay=weight_decay, 
                          momentum=momentum, 
                          foreach=True)

eval_plugin = EvaluationPlugin(
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    cpu_usage_metrics(experience=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    gpu_usage_metrics(gpu_id=0, minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger, wandb_logger]
)

In [11]:
from avalanche.training import Naive
cl_strategy = Naive(
    model=model, 
    optimizer=optimizer,
    criterion=CrossEntropyLoss(), 
    train_mb_size=batch_size, 
    train_epochs=epochs,
    device="cuda:0",
    eval_mb_size=batch_size,
    evaluator=eval_plugin)



In [12]:
# TRAINING LOOP
print('Starting experiment...')
results = []
for i, experience in enumerate(bm.train_stream):
# if True:
    # print("Start of experience: ", experience.current_experience)
    # print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(bm.test_stream[i]))

Starting experiment...
-- >> Start of training phase << --
100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:53<00:00,  7.61s/it]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream = 53997.1387
	DiskUsage_MB/train_phase/train_stream = 53997.1387
	Loss_Epoch/train_phase/train_stream = 1.1706
	Loss_MB/train_phase/train_stream = 0.9879
	MaxGPU0Usage_Epoch/train_phase/train_stream = 100.0000
	MaxGPU0Usage_MB/train_phase/train_stream = 20.0000
	RunningTime_Epoch/train_phase/train_stream = 1.5810
	Time_Epoch/train_phase/train_stream = 53.0761
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 from test stream --
100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:12<00:00,  1.78s/it]
> Eval on experience 0 from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Exp000 = 41.4229
	D

In [13]:
import pickle
pickle.dump(results,open(f"./log/eval/{func_name}.pkl","wb"))

In [14]:
wandb.finish()

CPUUsage_Exp/eval_phase/test_stream/Exp000,▁
CPUUsage_Exp/eval_phase/test_stream/Exp001,▁
CPUUsage_Exp/eval_phase/test_stream/Exp002,▁
CPUUsage_Exp/eval_phase/test_stream/Exp003,▁
CPUUsage_Exp/eval_phase/test_stream/Exp004,▁
DiskUsage_Epoch/train_phase/train_stream,█▁▁▂▃
DiskUsage_Exp/eval_phase/test_stream/Exp000,▁
DiskUsage_Exp/eval_phase/test_stream/Exp001,▁
DiskUsage_Exp/eval_phase/test_stream/Exp002,▁
DiskUsage_Exp/eval_phase/test_stream/Exp003,▁
DiskUsage_Exp/eval_phase/test_stream/Exp004,▁


## Train using EWC

In [15]:
func_name = "EWC"

In [16]:
model = UNet(n_channels=3, n_classes=4)

In [17]:
# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger(f"./log/tb_data/{func_name}")

wandb_logger = WandBLogger(project_name="UNet_CL",
                           run_name=func_name,
                           path=f".\log\checkpoint\{func_name}",
                           dir=f".\log\wandb\{func_name}")

# log to text file
text_logger = TextLogger(open(f'./log/{func_name}.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

wandb: WARNING Path .\log\wandb\EWC\wandb\ wasn't writable, using system temp directory.


In [18]:
# CREATE THE STRATEGY INSTANCE (EWC)
optimizer = optim.RMSprop(model.parameters(),
                          lr=learning_rate, 
                          weight_decay=weight_decay, 
                          momentum=momentum, 
                          foreach=True)

eval_plugin = EvaluationPlugin(
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    cpu_usage_metrics(experience=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    gpu_usage_metrics(gpu_id=0, minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger, wandb_logger]
)

In [19]:
from avalanche.training import EWC
cl_strategy = EWC(
    model=model, 
    optimizer=optimizer,
    criterion=CrossEntropyLoss(), 
    ewc_lambda=0.005,
    train_mb_size=batch_size, 
    train_epochs=epochs,
    device="cuda:0",
    eval_mb_size=batch_size,
    evaluator=eval_plugin)




In [20]:
# TRAINING LOOP
print('Starting experiment...')
results = []
for i, experience in enumerate(bm.train_stream):
# if True:
    # print("Start of experience: ", experience.current_experience)
    # print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(bm.test_stream[i]))

Starting experiment...
-- >> Start of training phase << --
100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:50<00:00,  7.28s/it]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream = 53975.8818
	DiskUsage_MB/train_phase/train_stream = 53975.8818
	Loss_Epoch/train_phase/train_stream = 1.1765
	Loss_MB/train_phase/train_stream = 1.0125
	MaxGPU0Usage_Epoch/train_phase/train_stream = 100.0000
	MaxGPU0Usage_MB/train_phase/train_stream = 43.0000
	RunningTime_Epoch/train_phase/train_stream = 1.1928
	Time_Epoch/train_phase/train_stream = 50.7457
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 from test stream --
100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:10<00:00,  1.43s/it]
> Eval on experience 0 from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Exp000 = 47.6060
	D

In [21]:
import pickle
pickle.dump(results,open(f"./log/eval/{func_name}.pkl","wb"))

In [22]:
wandb.finish()

CPUUsage_Exp/eval_phase/test_stream/Exp000,▁
CPUUsage_Exp/eval_phase/test_stream/Exp001,▁
CPUUsage_Exp/eval_phase/test_stream/Exp002,▁
CPUUsage_Exp/eval_phase/test_stream/Exp003,▁
CPUUsage_Exp/eval_phase/test_stream/Exp004,▁
DiskUsage_Epoch/train_phase/train_stream,▁▄▆▇█
DiskUsage_Exp/eval_phase/test_stream/Exp000,▁
DiskUsage_Exp/eval_phase/test_stream/Exp001,▁
DiskUsage_Exp/eval_phase/test_stream/Exp002,▁
DiskUsage_Exp/eval_phase/test_stream/Exp003,▁
DiskUsage_Exp/eval_phase/test_stream/Exp004,▁


## Train using LFL

In [23]:
func_name = "LFL"

In [24]:
model = UNet(n_channels=3, n_classes=4)

In [25]:
# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger(f"./log/tb_data/{func_name}")

wandb_logger = WandBLogger(project_name="UNet_CL",
                           run_name=func_name,
                           path=f".\log\checkpoint\{func_name}",
                           dir=f".\log\wandb\{func_name}")

# log to text file
text_logger = TextLogger(open(f'./log/{func_name}.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

wandb: WARNING Path .\log\wandb\LFL\wandb\ wasn't writable, using system temp directory.


In [26]:
# CREATE THE STRATEGY INSTANCE (LFL)
optimizer = optim.RMSprop(model.parameters(),
                          lr=learning_rate, 
                          weight_decay=weight_decay, 
                          momentum=momentum, 
                          foreach=True)

eval_plugin = EvaluationPlugin(
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    cpu_usage_metrics(experience=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    gpu_usage_metrics(gpu_id=0, minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger, wandb_logger]
)

In [27]:
from utils.util import SGLFL
cl_strategy = SGLFL(
    model=model, 
    optimizer=optimizer,
    criterion=CrossEntropyLoss(), 
    lambda_e=0.005,
    train_mb_size=batch_size, 
    train_epochs=epochs,
    device="cuda:0",
    eval_mb_size=batch_size,
    evaluator=eval_plugin)



In [28]:
# TRAINING LOOP
print('Starting experiment...')
results = []
for i, experience in enumerate(bm.train_stream):
# if True:
    # print("Start of experience: ", experience.current_experience)
    # print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(bm.test_stream[i]))

Starting experiment...
-- >> Start of training phase << --
100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:50<00:00,  7.28s/it]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream = 54019.9463
	DiskUsage_MB/train_phase/train_stream = 54019.9463
	Loss_Epoch/train_phase/train_stream = 1.2492
	Loss_MB/train_phase/train_stream = 1.1023
	MaxGPU0Usage_Epoch/train_phase/train_stream = 100.0000
	MaxGPU0Usage_MB/train_phase/train_stream = 43.0000
	RunningTime_Epoch/train_phase/train_stream = 1.2026
	Time_Epoch/train_phase/train_stream = 50.7797
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 from test stream --
100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:09<00:00,  1.42s/it]
> Eval on experience 0 from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Exp000 = 44.6325
	D

In [29]:
import pickle
pickle.dump(results,open(f"./log/eval/{func_name}.pkl","wb"))

In [30]:
wandb.finish()

CPUUsage_Exp/eval_phase/test_stream/Exp000,▁
CPUUsage_Exp/eval_phase/test_stream/Exp001,▁
CPUUsage_Exp/eval_phase/test_stream/Exp002,▁
CPUUsage_Exp/eval_phase/test_stream/Exp003,▁
CPUUsage_Exp/eval_phase/test_stream/Exp004,▁
DiskUsage_Epoch/train_phase/train_stream,▁▅▆▇█
DiskUsage_Exp/eval_phase/test_stream/Exp000,▁
DiskUsage_Exp/eval_phase/test_stream/Exp001,▁
DiskUsage_Exp/eval_phase/test_stream/Exp002,▁
DiskUsage_Exp/eval_phase/test_stream/Exp003,▁
DiskUsage_Exp/eval_phase/test_stream/Exp004,▁


## Train using SI

In [31]:
func_name = "SI"

In [32]:
model = UNet(n_channels=3, n_classes=4)

In [33]:
# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger(f"./log/tb_data/{func_name}")

wandb_logger = WandBLogger(project_name="UNet_CL",
                           run_name=func_name,
                           path=f".\log\checkpoint\{func_name}",
                           dir=f".\log\wandb\{func_name}")

# log to text file
text_logger = TextLogger(open(f'./log/{func_name}.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

wandb: WARNING Path .\log\wandb\SI\wandb\ wasn't writable, using system temp directory.


In [34]:
# CREATE THE STRATEGY INSTANCE (SI)
optimizer = optim.RMSprop(model.parameters(),
                          lr=learning_rate, 
                          weight_decay=weight_decay, 
                          momentum=momentum, 
                          foreach=True)

eval_plugin = EvaluationPlugin(
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    cpu_usage_metrics(experience=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    gpu_usage_metrics(gpu_id=0, minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger, wandb_logger]
)

In [35]:
from avalanche.training import SynapticIntelligence
cl_strategy = SynapticIntelligence(
    model=model, 
    optimizer=optimizer,
    criterion=CrossEntropyLoss(), 
    si_lambda=0.005,
    train_mb_size=batch_size, 
    train_epochs=epochs,
    device="cuda:0",
    eval_mb_size=batch_size,
    evaluator=eval_plugin)

In [36]:
# TRAINING LOOP
print('Starting experiment...')
results = []
for i, experience in enumerate(bm.train_stream):
# if True:
    # print("Start of experience: ", experience.current_experience)
    # print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(bm.test_stream[i]))

Starting experiment...
-- >> Start of training phase << --
100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [01:00<00:00,  8.62s/it]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream = 54072.6855
	DiskUsage_MB/train_phase/train_stream = 54072.6855
	Loss_Epoch/train_phase/train_stream = 1.0782
	Loss_MB/train_phase/train_stream = 0.9304
	MaxGPU0Usage_Epoch/train_phase/train_stream = 100.0000
	MaxGPU0Usage_MB/train_phase/train_stream = 1.0000
	RunningTime_Epoch/train_phase/train_stream = 1.4543
	Time_Epoch/train_phase/train_stream = 60.1733
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 from test stream --
100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:15<00:00,  2.18s/it]
> Eval on experience 0 from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Exp000 = 54.4045
	Di

In [37]:
import pickle
pickle.dump(results,open(f"./log/eval/{func_name}.pkl","wb"))

In [38]:
wandb.finish()

CPUUsage_Exp/eval_phase/test_stream/Exp000,▁
CPUUsage_Exp/eval_phase/test_stream/Exp001,▁
CPUUsage_Exp/eval_phase/test_stream/Exp002,▁
CPUUsage_Exp/eval_phase/test_stream/Exp003,▁
CPUUsage_Exp/eval_phase/test_stream/Exp004,▁
DiskUsage_Epoch/train_phase/train_stream,▁▂▅▇█
DiskUsage_Exp/eval_phase/test_stream/Exp000,▁
DiskUsage_Exp/eval_phase/test_stream/Exp001,▁
DiskUsage_Exp/eval_phase/test_stream/Exp002,▁
DiskUsage_Exp/eval_phase/test_stream/Exp003,▁
DiskUsage_Exp/eval_phase/test_stream/Exp004,▁


## Train using GEM

In [39]:
func_name = "GEM"

In [40]:
model = UNet(n_channels=3, n_classes=4)

In [41]:
# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger(f"./log/tb_data/{func_name}")

wandb_logger = WandBLogger(project_name="UNet_CL",
                           run_name=func_name,
                           path=f".\log\checkpoint\{func_name}",
                           dir=f".\log\wandb\{func_name}")

# log to text file
text_logger = TextLogger(open(f'./log/{func_name}.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

wandb: WARNING Path .\log\wandb\GEM\wandb\ wasn't writable, using system temp directory.


In [42]:
# CREATE THE STRATEGY INSTANCE (GEM)
optimizer = optim.RMSprop(model.parameters(),
                          lr=learning_rate, 
                          weight_decay=weight_decay, 
                          momentum=momentum, 
                          foreach=True)

eval_plugin = EvaluationPlugin(
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    cpu_usage_metrics(experience=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    gpu_usage_metrics(gpu_id=0, minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger, wandb_logger]
)


In [43]:
from avalanche.training import GEM
cl_strategy = GEM(
    model=model, 
    optimizer=optimizer,
    criterion=CrossEntropyLoss(), 
    patterns_per_exp=10,
    train_mb_size=batch_size, 
    train_epochs=epochs,
    device="cuda:0",
    eval_mb_size=batch_size,
    evaluator=eval_plugin)

In [44]:
# TRAINING LOOP
print('Starting experiment...')
results = []
for i, experience in enumerate(bm.train_stream):
# if True:
    # print("Start of experience: ", experience.current_experience)
    # print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(bm.test_stream[i]))

Starting experiment...
-- >> Start of training phase << --
100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:59<00:00,  8.55s/it]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream = 54112.4238
	DiskUsage_MB/train_phase/train_stream = 54112.4238
	Loss_Epoch/train_phase/train_stream = 0.9938
	Loss_MB/train_phase/train_stream = 0.8568
	MaxGPU0Usage_Epoch/train_phase/train_stream = 100.0000
	MaxGPU0Usage_MB/train_phase/train_stream = 6.0000
	RunningTime_Epoch/train_phase/train_stream = 1.4302
	Time_Epoch/train_phase/train_stream = 59.6464
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 from test stream --
100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:15<00:00,  2.18s/it]
> Eval on experience 0 from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Exp000 = 50.0074
	Di

In [45]:
import pickle
pickle.dump(results,open(f"./log/eval/{func_name}.pkl","wb"))

In [46]:
wandb.finish()

CPUUsage_Exp/eval_phase/test_stream/Exp000,▁
CPUUsage_Exp/eval_phase/test_stream/Exp001,▁
CPUUsage_Exp/eval_phase/test_stream/Exp002,▁
CPUUsage_Exp/eval_phase/test_stream/Exp003,▁
CPUUsage_Exp/eval_phase/test_stream/Exp004,▁
DiskUsage_Epoch/train_phase/train_stream,▁▃▅▆█
DiskUsage_Exp/eval_phase/test_stream/Exp000,▁
DiskUsage_Exp/eval_phase/test_stream/Exp001,▁
DiskUsage_Exp/eval_phase/test_stream/Exp002,▁
DiskUsage_Exp/eval_phase/test_stream/Exp003,▁
DiskUsage_Exp/eval_phase/test_stream/Exp004,▁
